In [1]:
%run Main.ipynb

# Redshift Drift 

## - Spectroscopic velocity

$\Delta v = \frac{c \Delta z}{1+z} = c H_0 \Delta t \left[1 - \frac{E(z)}{1+z}\right] = kh \left[1 - \frac{E(z)}{1+z}\right]$, where $H_0 = 100 H_{100}$ and $k = c H_{100}\delta t$

Assuming $\delta t = 1$ year we have $k=3.064$ cm/s.

In [2]:
def spect_velocity(zs, h, OmegaM, w0, wa, NrYears):
    if w0 == None: 
        w0 = -1.
    if wa == None: 
        wa = 0
    return c100t1 * NrYears * h * ( 1 - 1/(1+zs) * function_E_z(zs, OmegaM, w0, wa))

## - Spectroscopic velocity uncertainty

$\sigma_{\Delta v} = 1.35 \frac{2370}{S/N} \sqrt{\frac{30}{N_{QSO}})} \left({\frac{5}{1+z_{QSO}}}\right)^x$, with $x=1.7$ for $z\leqslant4$ and $x=0.9$ for $z>4$ 

In [3]:
def uncLiske_spect_velocity(zs):
    NrQSO = 6
    SN    = 3000
    unc_zELT = np.zeros_like(zs)
    for i in np.arange(len(zs)):
        if zs[i] <= 4:
            x = 1.7
        else:
            x = .9
        unc_zELT[i] = 1.35 * 2370/SN * np.sqrt(30/NrQSO) * (5/(1+zs[i]))**x
    return unc_zELT

In [4]:
def uncSKA_spect_velocity(zs, h, OmegaM, w0, wa, NrYears):
    NrZs = len(zs)
    Percent = np.array([.02, .04, .06, .08, .1])
    unc_zSKA = np.zeros(NrZs)
    for i in np.arange(NrZs):
        unc_zSKA[i] = Percent[i] * spect_velocity(zs[i], h, OmegaM, w0, wa, NrYears)
    return unc_zSKA

## - Derivatives calculation

In [5]:
def dfdh_z(z, h, OmegaM, w0, wa, NrYears):
    return ( spect_velocity(z, h+hh, OmegaM, w0, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh

def dfdOmegaM_z(z, h, OmegaM, w0, wa, NrYears):
    return ( spect_velocity(z, h,OmegaM+hh, w0, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh

def dfdw0_z(z, h, OmegaM, w0, wa, NrYears):
    if w0 != None:
        return ( spect_velocity(z, h, OmegaM, w0+hh, wa, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh
    else:
        return 0

def dfdwa_z(z, h, OmegaM, w0, wa, NrYears):
    if wa != None:
        return ( spect_velocity(z, h,OmegaM, w0, wa+hh, NrYears)-spect_velocity(z, h, OmegaM, w0, wa, NrYears) )/hh
    else:
        return 0

## - Add Priors (Separate, Planck and COrE)

In [6]:
def add1DPriors(F, priors): #to add priors locally in the intended parameters
    '''F-Fisher matrix, priors-[[sigma_prior1,local1],...]'''
    G = np.copy(F)
    for i in range(len(priors)): 
        G[priors[i][1],priors[i][1]]+=priors[i][0]**-2
    return G

def addDimPrior_z(Foriginal_z, Fprior_z): #to add the Planck and COrE priors
    'Fori - original F, Fprior - F with the priors but different dimension'
    dimFreal = len(Foriginal_z)
    while len(Fprior_z) != dimFreal:
        Fprior_z = np.insert(Fprior_z, len(Fprior_z), 0, axis=1) # add column
        Fprior_z = np.insert(Fprior_z, len(Fprior_z), 0, axis=0) # add line
    return Foriginal_z+Fprior_z

def addPriors_z(F_z, cur_z): #combination of the previous two
    'F-Fisher matrix, cur=1 -current priors, cur=0-future priors'
    newF_z = np.copy(F_z)
    if len(F_z) == 2: # 2x2 matrix - minimum
        if cur_z == 1:
            newF_z = addDimPrior_z(F_z, FPlanck)
        else:
            newF_z = addDimPrior_z(F_z, FCORE)
    elif len(F_z) == 3: # 3x3 matrix
        if cur_z == 1:
            newF_z = addDimPrior_z(F_z, FPlanck)
            newF_z = add1DPriors(newF_z, [[uncw0Cur,2]])
        else:
            newF_z = addDimPrior_z(F_z, FCORE)
            newF_z = add1DPriors(newF_z, [[uncw0Fut,2]])
    elif len(F_z) == 4: # 4x4 matrix - maximum
        if cur_z == 1:
            newF_z = addDimPrior_z(F_z, FPlanck)
            newF_z = add1DPriors(newF_z, [[uncw0Cur,2],[uncwaCur,3]])
        else:
            newF_z = addDimPrior_z(F_z, FCORE)
    return newF_z

In [7]:
#Priors in h and OmegaM for Planck and CORE
def getPriorsHOmegaM(Priors): # the 1st row in h and the second Om
    return np.array([[(2*h*OmegaM)**2*Priors**-2   , 2*OmegaM*h**3*Priors**-2],
                     [2*OmegaM*h**3*Priors**-2 , h**4*Priors**-2]])

In [8]:
uncw0Cur = .1
uncwaCur = .3
uncPlack = .0013 # uncertainty in Omega_m * h**2
FPlanck  = getPriorsHOmegaM(uncPlack)

In [9]:
uncw0Fut = .02
uncwaFut = .1
uncCORE  = .00028 # uncertainty in Omega_m * h**2
FCORE    = getPriorsHOmegaM(uncCORE)

# Measurements
### - Redshifts

In [10]:
zLiske = np.array([2.,  2.5, 3.,  3.5, 4.5])
zSKA   = np.array([0.1, 0.2, 0.3, 0.4, 0.5])
zGoldenSample = np.array([4.111, 4.260, 3.240, 4.072, 3.655, 2.800, 3.835, 4.728, 3.400, 3.123, 4.147, 4.040, 3.093, 4.190, 3.370, 3.765, 3.765, 4.780, 3.700, 3.273, 3.090, 3.703, 3.712, 3.783, 3.685, 3.549, 3.897, 3.280, 3.473, 4.771])

### - Uncertainties

In [11]:
unc_zLiske = uncLiske_spect_velocity(zLiske)
unc_zSKA   = uncSKA_spect_velocity(zSKA, h, OmegaM, w0=-1, wa=0, NrYears=.5)
unc_zGoldenSample = np.zeros(30) + 22.8

# Fisher Matrices
## - No Priors
#### $\Lambda$CDM

In [12]:
FM_LCDM_Liske_np_z  = FisherMatrix_z(zLiske, unc_zLiske, h, OmegaM, w0=None, wa=None, NrYears=20, analysis = 'redshift')
FM_LCDM_SKA_np_z    = FisherMatrix_z(zSKA , unc_zSKA, h, OmegaM, w0=None, wa=None, NrYears=.5, analysis = 'redshift')
FM_LCDM_GoldenSample_np_z = FisherMatrix_z(zGoldenSample, unc_zGoldenSample, h, OmegaM, w0=None, wa=None, NrYears=25, analysis = 'redshift')

FM_LCDM_LiskeSKA_np_z = FM_LCDM_Liske_np_z + FM_LCDM_SKA_np_z
FM_LCDM_GoldenSKA_np_z = FM_LCDM_GoldenSample_np_z + FM_LCDM_SKA_np_z

#### CPL

In [13]:
FM_CPL_Liske_np_z = FisherMatrix_z(zLiske, unc_zLiske, h, OmegaM, w0=-0.9, wa=0.3, NrYears=20, analysis = 'redshift')
FM_CPL_SKA_np_z   = FisherMatrix_z(zSKA, unc_zSKA, h, OmegaM, w0=-0.9, wa=0.3, NrYears=.5, analysis = 'redshift')
FM_CPL_GoldenSample_np_z = FisherMatrix_z(zGoldenSample, unc_zGoldenSample, h, OmegaM, w0=-0.9, wa=0.3, NrYears=25, analysis = 'redshift')

FM_CPL_LiskeSKA_np_z = FM_CPL_Liske_np_z + FM_CPL_SKA_np_z
FM_CPL_GoldenSKA_np_z = FM_CPL_GoldenSample_np_z + FM_CPL_SKA_np_z

## - With Priors
#### $\Lambda$CDM

In [14]:
#Priors in h and OmegaM
FM_LCDM_Liske_pOH = add1DPriors(FM_LCDM_Liske_np_z, [[unch,0],[uncOmegaM,1]])
FM_LCDM_SKA_pOH = add1DPriors(FM_LCDM_SKA_np_z, [[unch,0],[uncOmegaM,1]])
FM_LCDM_GoldenSample_pOH = add1DPriors(FM_LCDM_GoldenSample_np_z, [[unch,0],[uncOmegaM,1]])

PriorsOnly = np.zeros((2,2))
PriorsOnly = add1DPriors(PriorsOnly, [[unch,0],[uncOmegaM,1]])

FM_LCDM_LiskeSKA_pOH = FM_LCDM_Liske_pOH + FM_LCDM_SKA_pOH
FM_LCDM_GoldenSKA_pOH = FM_LCDM_GoldenSample_pOH + FM_LCDM_SKA_pOH

In [15]:
#Current Priors cur_z=1 - Planck
FM_LCDM_Liske_pCur_z = addPriors_z(FM_LCDM_Liske_np_z, cur_z=1)
FM_LCDM_SKA_pCur_z   = addPriors_z(FM_LCDM_SKA_np_z  , cur_z=1)
FM_LCDM_GoldenSample_pCur_z = addPriors_z(FM_LCDM_GoldenSample_np_z, cur_z=1)

FM_LCDM_LiskeSKA_pCur_z = addPriors_z(FM_LCDM_LiskeSKA_np_z, cur_z=1)
FM_LCDM_GoldenSKA_pCur_z = addPriors_z(FM_LCDM_GoldenSKA_np_z, cur_z=1)

#### CPL

In [16]:
#Priors in h,OmegaM,w0,wa 
FM_CPL_Liske_pOH = add1DPriors(FM_CPL_Liske_np_z, [[unch,0],[uncOmegaM,1],[uncw0Cur,2], [uncwaCur,3]])
FM_CPL_SKA_pOH = add1DPriors(FM_CPL_SKA_np_z, [[unch,0],[uncOmegaM,1],[uncw0Cur,2], [uncwaCur,3]])
FM_CPL_GoldenSample_pOH = add1DPriors(FM_CPL_GoldenSample_np_z, [[unch,0],[uncOmegaM,1],[uncw0Cur,2], [uncwaCur,3]])

FM_CPL_LiskeSKA_pOH = FM_CPL_Liske_pOH + FM_CPL_SKA_pOH
FM_CPL_GoldenSKA_pOH = FM_CPL_GoldenSample_pOH + FM_CPL_SKA_pOH

# Confidence ellipses (1-sigma)
#### $\Lambda$CDM

In [17]:
def grafhOm(h,OmegaM,w0,wa,Fs,*model): 
    ells,nn = [],[]
    if model != () :
        nn.append('_'+model[0]+'_')
    for i in range(len(Fs)):
        try: # if the matrix is 4x4, marginalizes wa
            Fs[i][3][3]
            Fm = marginalize(Fs[i],'wa',analysis='redshift')
        except IndexError:
            Fm = np.copy(Fs[i])
        try: # if the matrix is 3x3, marginalizes w0
            Fm[2][2]
            Fm = marginalize(Fm,'w0',analysis='redshift')
        except IndexError:
            pass
        Pa = preelipse(Fm)
        color,name,style = colornamestyle(Fs[i])
        ells.append([h,OmegaM,Pa[0],Pa[1],Pa[2],name,color,style])
        if name[-3:] not in nn:
            nn.append(name[-3:])
    ellipses(ells,['h','$\Omega_m$'])
    plt.xlim(-1.2,2.6)
    plt.ylim(0.0,0.6)
    plt.tick_params(axis='both', which='major', labelsize=12)
    #   plt.savefig('hOm.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#grafhOm(h,OmegaM,w0,wa,np.array([FM_LCDM_Liske_np_z, FM_LCDM_SKA_np_z,FM_LCDM_GoldenSample_np_z]))

#### CPL

In [18]:
def CPLRedwaw0(w0,wa,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'OmegaM',analysis='redshift')
        Fm = marginalize(Fm,'h',analysis='redshift')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle(Fs[i])
        ells.append([w0,wa, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$w_0$','$w_a$'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    #plt.savefig('CPLRedwaw0.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#CPLRedwaw0(-0.9,0.3,[FM_CPL_Liske_pOH,FM_CPL_SKA_pOH,FM_CPL_GoldenSample_pOH])

In [19]:
def CPLRedOmegaMw0(OmegaM,w0,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'wa',analysis='redshift')
        Fm = marginalize(Fm,'h',analysis='redshift')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle(Fs[i])
        ells.append([OmegaM,w0, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$\Omega_m$','$w_0$'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    #plt.savefig('CPLOmegaMwa.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#CPLRedOmegaMw0(0.3,-0.9,[FM_CPL_Liske_pOH,FM_CPL_SKA_pOH,FM_CPL_GoldenSample_pOH])

In [20]:
def CPLRedOmegaMh(OmegaM,h,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'wa',analysis='redshift')
        Fm = marginalize(Fm,'w0',analysis='redshift')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle(Fs[i])
        ells.append([h,OmegaM, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$h$','$\Omega_m$'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.xlim(0.6,0.8)
    plt.ylim(0.25,0.35)
    #plt.savefig('CPLRedOmegaMh.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#CPLRedOmegaMh(0.3,0.7,[FM_CPL_Liske_pOH,FM_CPL_SKA_pOH,FM_CPL_GoldenSample_pOH])

In [21]:
def CPLRedOmegaMwa(OmegaM,wa,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'w0',analysis='redshift')
        Fm = marginalize(Fm,'h',analysis='redshift')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle(Fs[i])
        ells.append([OmegaM,wa, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$\Omega_m$','$w_a$'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    #plt.savefig('CPLOmegaMwa.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#CPLRedOmegaMwa(0.3,0.3,[FM_CPL_Liske_pOH,FM_CPL_SKA_pOH,FM_CPL_GoldenSample_pOH])

In [22]:
def CPLRedhwa(h,wa,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'w0',analysis='redshift')
        Fm = marginalize(Fm,'OmegaM',analysis='redshift')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle(Fs[i])
        ells.append([wa,h, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$w_a$','h'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    #plt.savefig('CPLOmegaMwa.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#CPLRedhwa(0.7,0.3,[FM_CPL_Liske_pOH,FM_CPL_SKA_pOH,FM_CPL_GoldenSample_pOH])

In [23]:
def CPLRedhw0(h,w0,Fs): 
    ells = []
    for i in range(len(Fs)):
        Fm = marginalize(Fs[i],'wa',analysis='redshift')
        Fm = marginalize(Fm,'OmegaM',analysis='redshift')
        Pa = preelipse(Fm)
        color,name,style = colornamestyle(Fs[i])
        ells.append([w0,h, Pa[0],Pa[1],Pa[2],name,color,style])
    ellipses(ells,['$w_0$','h'])
    plt.tick_params(axis='both', which='major', labelsize=12)
    #plt.savefig('CPLOmegaMwa.png', dpi=300, bbox_inches='tight')
    plt.show()
    
#CPLRedhw0(0.7,-0.9,[FM_CPL_Liske_pOH,FM_CPL_SKA_pOH,FM_CPL_GoldenSample_pOH])

# FoM vs $\sigma_v$ for the Golden Sample analysis

In [24]:
sigma = np.arange(0.9, 22.8+0.1, 0.1)   
FOM = np.zeros(len(sigma))
RHO = np.zeros(len(sigma))

for i in range(len(sigma)):
    sigmaV = np.zeros(30) + sigma[i]
    FOM_aux = FisherMatrix_z(zGoldenSample, sigmaV, h, OmegaM, w0=None, wa=None, NrYears=25, analysis = 'redshift')
    FOM[i], RHO[i] = FigMeriteRho(FOM_aux,[],analysis='redshift')

In [25]:
sigma2 = np.arange(0.9, 22.8+0.1, 0.01)   
FOM2 = np.zeros(len(sigma2))
RHO2 = np.zeros(len(sigma2))

for i in range(len(sigma2)):
    sigmaV2 = np.zeros(30) + sigma2[i]
    FOM_aux2 = FisherMatrix_z(zGoldenSample, sigmaV2, h, OmegaM, w0=None, wa=None, NrYears=25, analysis = 'redshift')
    FOM_aux3 = add1DPriors(FOM_aux2, [[unch,0],[uncOmegaM,1]])
    FOM2[i], RHO2[i] = FigMeriteRho(FOM_aux3,[],analysis='redshift')

In [26]:
sigma3 = np.arange(0.9, 22.8+0.1, 0.01)   
FOM3 = np.zeros(len(sigma3))
RHO3 = np.zeros(len(sigma3))

for i in range(len(sigma3)):
    sigmaV3 = np.zeros(30) + sigma3[i]
    FOM_aux4 = FisherMatrix_z(zGoldenSample, sigmaV3, h, OmegaM, w0=None, wa=None, NrYears=25, analysis = 'redshift')
    FOM_aux5 = addPriors_z(FOM_aux4, cur_z=1)
    FOM3[i], RHO3[i] = FigMeriteRho(FOM_aux5,[],analysis='redshift')

In [27]:
def intercept(sigma, FOM, value):
    interp = InterpolatedUnivariateSpline(sigma, FOM - value)
    root = interp.roots()
    return root[0], interp(root[0])  + value

#### No Priors

In [28]:
fig = plt.figure(figsize=[10, 8])

target_value = FigMeriteRho(FM_LCDM_Liske_np_z,[],analysis='redshift')[0]

plt.plot(sigma, FOM, color = 'purple')
plt.hlines(target_value, sigma[0], sigma[-1], color = 'blue')
plt.ylabel('FoM', fontsize=16)
plt.xlabel(r'$\sigma_v$',fontsize=16)

x_pos, y_pos = intercept(sigma, FOM, target_value)

my_formatter = "{0:.2f}"
my_formatter2 = "{:.0f}"
x_pos, y_pos = float(x_pos), float(y_pos)
plt.text (x_pos, y_pos+8,'({}, {})'. format(my_formatter.format(x_pos), my_formatter2.format(y_pos)), fontsize=12)

plt.scatter(x_pos, y_pos, color = 'black')

plt.ylim(0,400)
plt.tick_params(axis='both', which='major', labelsize=12)
#Comment to display the graphic:
plt.close()
#plt.savefig('a.png', dpi=300, bbox_inches='tight')

#### Separate Priors

In [29]:
fig = plt.figure(figsize=[10, 8])
plt.ylabel('FoM', fontsize=16)
plt.xlabel(r'$\sigma_v$',fontsize=16)

target_value = FigMeriteRho(FM_LCDM_Liske_pOH,[],analysis='redshift')[0]
plt.plot (sigma2, FOM2, color = 'purple')
plt.hlines(target_value, sigma2[0], sigma2[-1], color = 'blue')

my_formatter = "{0:.2f}"
my_formatter2 = "{:.0f}"

x_pos2, y_pos2 = intercept(sigma2, FOM2, target_value)
x_pos2, y_pos2 = float(x_pos2), float(y_pos2)
plt.text (x_pos2, y_pos2+25,'({}, {})'. format(my_formatter.format(x_pos2), my_formatter2.format(y_pos2)), fontsize=12)

plt.scatter(x_pos2, y_pos2, color = 'black')

plt.ylim(200,2000)
plt.tick_params(axis='both', which='major', labelsize=12)
#Comment to display the graphic:
plt.close()
#plt.savefig('b.png', dpi=300, bbox_inches='tight')

#### Planck Prior

In [30]:
fig = plt.figure(figsize=[10, 8])
plt.ylabel('FoM', fontsize=16)
plt.xlabel(r'$\sigma_v$',fontsize=16)

target_value = FigMeriteRho(FM_LCDM_Liske_pCur_z,[],analysis='redshift')[0]
plt.plot (sigma3, FOM3, color = 'purple')
plt.hlines(target_value, sigma3[0], sigma3[-1], color = 'blue')

my_formatter = "{0:.2f}"
my_formatter2 = "{:.0f}"

x_pos3, y_pos3 = intercept(sigma3, FOM3, target_value)
x_pos3, y_pos3 = float(x_pos3), float(y_pos3)
plt.text (x_pos3, y_pos3+300,'({}, {})'. format(my_formatter.format(x_pos3), my_formatter2.format(y_pos3)), fontsize=12)

plt.scatter(x_pos3, y_pos3, color = 'black')

plt.ylim(2000,20000)
plt.tick_params(axis='both', which='major', labelsize=12)
#Comment to display the graphic:
plt.close()
#plt.savefig('c.png', dpi=300, bbox_inches='tight')

# Tables
#### $\Lambda$CDM

In [31]:
def tableLambdaCDM(matrices):
    i=0
    df=pd.DataFrame()
    nformat = "{:.0f}"

    for matrix in matrices:
        columns = {'ρ(h, Ωm)':FigMeriteRho(matrix,[],analysis='redshift')[1], 
                  'FoM(h, Ωm)': nformat.format(FigMeriteRho(matrix,[],analysis='redshift')[0]), 
                  'σ(h)':Sigmas(matrix)[0],
                  'σ(Ωm)':Sigmas(matrix)[1]}
        i+=1
        df = df.append(columns, ignore_index=True)
        del columns
    Priors = ['None', 'Separated', 'Planck', 'None', 'Separated', 'Planck', 'None', 'Separated', 'Planck', 'None', 'Separated', 'Planck', 'None', 'Separated', 'Planck']
    Datasets = ['','Liske', '', '','Golden Sample', '', '', 'SKA','', '', 'Liske+SKA','', '', 'Golden Sample + SKA', '']
    df.insert(0, "Datasets", Datasets, True)
    df.insert(1, 'Priors', Priors, True)
    
    return df

In [32]:
#ΛCDM - Liske + Golden Sample + SKA + (Liske+SKA) + (GoldenSample + SKA)

#matrices = [FM_LCDM_Liske_np_z, FM_LCDM_Liske_pOH, FM_LCDM_Liske_pCur_z,
#            FM_LCDM_GoldenSample_np_z, FM_LCDM_GoldenSample_pOH, FM_LCDM_GoldenSample_pCur_z,
#            FM_LCDM_SKA_np_z, FM_LCDM_SKA_pOH, FM_LCDM_SKA_pCur_z,  
#            FM_LCDM_LiskeSKA_np_z, FM_LCDM_LiskeSKA_pOH, FM_LCDM_LiskeSKA_pCur_z, 
#            FM_LCDM_GoldenSKA_np_z, FM_LCDM_GoldenSKA_pOH, FM_LCDM_GoldenSKA_pCur_z]
#tab1 = tableLambdaCDM(matrices)

#See the table in DataFrame:
#tab1

#Transform dataframe to latex code:
#print(tab1.to_latex(index=False, float_format="%.3f"))

#### CPL

In [33]:
def tableCPL(matrices):
    i=0
    df=pd.DataFrame()
    nformat = "{:.0f}"
    float_format="{:.3f}"
    

    for matrix in matrices:
        rows = {'ρ(h, Ωm)':float_format.format(FigMeriteRho(matrix,['w0','wa'],analysis='redshift')[1]),
                'ρ(Ωm, w0)':float_format.format(FigMeriteRho(matrix,['h','wa'],analysis='redshift')[1]),
                'ρ(h, w0)':float_format.format(FigMeriteRho(matrix,['OmegaM','wa'],analysis='redshift')[1]),                       
                'ρ(Ωm, wa)':float_format.format(FigMeriteRho(matrix,['h','w0'],analysis='redshift')[1]),
                'ρ(h, wa)':float_format.format(FigMeriteRho(matrix,['OmegaM','w0'],analysis='redshift')[1]),
                'ρ(w0, wa)':float_format.format(FigMeriteRho(matrix,['h','OmegaM'],analysis='redshift')[1]),
                'FoM(h, Ωm)': nformat.format(FigMeriteRho(matrix,['w0','wa'],analysis='redshift')[0]),
                'FoM(Ωm, w0)': nformat.format(FigMeriteRho(matrix,['h','wa'],analysis='redshift')[0]),
                'FoM(h, w0)': nformat.format(FigMeriteRho(matrix,['OmegaM','wa'],analysis='redshift')[0]),
                'FoM(Ωm, wa)': nformat.format(FigMeriteRho(matrix,['h','w0'],analysis='redshift')[0]),
                'FoM(h, wa)': nformat.format(FigMeriteRho(matrix,['OmegaM','w0'],analysis='redshift')[0]),
                'FoM(w0, wa)': nformat.format(FigMeriteRho(matrix,['h','OmegaM'],analysis='redshift')[0]),
                'σ(h)':float_format.format(Sigmas(matrix)[0]),
                'σ(Ωm)':float_format.format(Sigmas(matrix)[1]),
                'σ(w0)':float_format.format(Sigmas(matrix)[2]),
                'σ(wa)':float_format.format(Sigmas(matrix)[3])}
        i+=1
        
        df = df.append(rows,ignore_index=True)
        del rows
    
    
    Datasets = ['Liske','Golden Sample', 'SKA', 'Liske+SKA', 'Golden Sample + SKA']
    df.insert(0, "Parameters", Datasets, True)
    
    df = df.transpose()
    
            
    return df

In [34]:
#CPL - Liske + Golden Sample + SKA + (Liske+SKA) + (GoldenSample + SKA)

#matricesCPL = [FM_CPL_Liske_pOH, FM_CPL_GoldenSample_pOH, FM_CPL_SKA_pOH, 
#               FM_CPL_LiskeSKA_pOH, FM_CPL_GoldenSKA_pOH]
#tab1 = tableCPL(matricesCPL)

#See the table in DataFrame:
#tab1

#Transform dataframe to latex code:
#print(tab1.to_latex(header=False,index=True))